In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("./data/tweets.csv", encoding='latin')
df['Date'] = pd.to_datetime(df['Date'], format='%a %b %d %H:%M:%S PDT %Y')
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

train_size = int(len(df_shuffled) * 0.7)
df = df_shuffled[:train_size]
test_set = df_shuffled[train_size:]

In [3]:
df = df[:100000]

In [4]:
try:
    df.drop(['ID', 'flag'], axis=1, inplace=True)
except KeyError:
    pass
df['Weekday'] = df['Date'].dt.weekday
df['Time'] = df['Date'].dt.time
df['Full_date'] = df['Date'].dt.date

df['Target'] = df['Target'].map({0: 0, 4: 1})
df['Length'] = df['Text'].apply(len)

import re


def count_hashtags(text):
    return len([c for c in text if c == '#'])


def count_mentions(text):
    return len([c for c in text if c == '@'])


def count_exclamation_marks(text):
    return len([c for c in text if c == '!'])


def detect_emoticons(text):
    emoticon_pattern = r'(:\)|:\(|;\)|:D|:P|:\||:\-\)|:\-\(|;\-\)|:‑D|:‑P|:‑\||<3)'
    return re.findall(emoticon_pattern, text)


df['Hashtags'] = df['Text'].apply(count_hashtags)
df['HasHashtags'] = df['Hashtags'] > 0
values_df = pd.DataFrame()
values_df['Hashtags'] = df['Hashtags'].value_counts()

df['Mentions'] = df['Text'].apply(count_mentions)
df['HasMentions'] = df['Mentions'] > 0
values_df['Mentions'] = df['Mentions'].value_counts()

df['ExclamationMarks'] = df['Text'].apply(count_exclamation_marks)
df['HasExclamationMarks'] = df['ExclamationMarks'] > 0
values_df['ExclamationMarks'] = df['ExclamationMarks'].value_counts()

df['Emoticons'] = df['Text'].apply(detect_emoticons)
emoticons = df[['Target', 'Emoticons']].explode('Emoticons').dropna().groupby(['Emoticons']).agg(
    ['mean', 'count']).reset_index()

In [5]:
import textstat

df['FRE'] = df['Text'].apply(lambda x: textstat.flesch_reading_ease(x))
df['GFI'] = df['Text'].apply(lambda x: textstat.gunning_fog(x))

In [6]:
from textblob import TextBlob


def analyze_textblob_sentiment(tweet):
    analysis = TextBlob(tweet)
    return analysis.sentiment.polarity, analysis.sentiment.subjectivity


df[['Polarity_TB', 'Subjectivity_TB']] = df['Text'].apply(lambda tweet: pd.Series(analyze_textblob_sentiment(tweet)))

In [7]:
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()


def analyze_vader_sentiment(tweet):
    scores = sia.polarity_scores(tweet)
    return scores['pos'], scores['neu'], scores['neg'], scores['compound']


df[['Positive_VADER', 'Neutral_VADER', 'Negative_VADER', 'Compound_VADER']] = df['Text'].apply(
    lambda tweet: pd.Series(analyze_vader_sentiment(tweet)))


In [8]:
df.head()

,Target,Date,User,Text,Weekday,Time,Full_date,Length,Hashtags,HasHashtags,...,HasExclamationMarks,Emoticons,FRE,GFI,Polarity_TB,Subjectivity_TB,Positive_VADER,Neutral_VADER,Negative_VADER,Compound_VADER
0,0,2009-06-25 00:52:47,timofuchs,@grazzly maaaaaate will you be at Glasto? Not ...,3,00:52:47,2009-06-25,67,0,False,...,False,[],82.81,2.2,0.00,0.000000,0.000,1.000,0.0,0.0000
1,1,2009-05-14 02:06:48,markdavidson,@Monkeylover35 Yeah. I had to get away for a b...,3,02:06:48,2009-05-14,93,0,False,...,False,[],96.69,3.4,0.50,0.500000,0.290,0.710,0.0,0.5994
2,1,2009-05-04 07:14:48,Maekii,The World is just amazing!,0,07:14:48,2009-05-04,27,0,False,...,True,[],100.24,2.0,0.75,0.900000,0.506,0.494,0.0,0.6239
3,0,2009-06-25 02:35:25,MissElisee,Just picked my sister up from work. she's al...,3,02:35:25,2009-06-25,100,0,False,...,False,[],95.17,4.0,0.00,0.000000,0.000,1.000,0.0,0.0000
4,0,2009-06-19 13:40:45,MissK20,I don't think changing my last name is meant t...,4,13:40:45,2009-06-19,52,0,False,...,True,[],102.61,4.4,0.00,0.066667,0.000,1.000,0.0,0.0000


In [10]:
df['Hour'] = df['Date'].dt.hour
df['skewed_hour_dist'] = df['Hour'].apply(lambda x: (16 - x) / 16 if x < 16 else (x - 16) / 8)
df.head()

,Target,Date,User,Text,Weekday,Time,Full_date,Length,Hashtags,HasHashtags,...,FRE,GFI,Polarity_TB,Subjectivity_TB,Positive_VADER,Neutral_VADER,Negative_VADER,Compound_VADER,Hour,skewed_hour_dist
0,0,2009-06-25 00:52:47,timofuchs,@grazzly maaaaaate will you be at Glasto? Not ...,3,00:52:47,2009-06-25,67,0,False,...,82.81,2.2,0.00,0.000000,0.000,1.000,0.0,0.0000,0,1.0000
1,1,2009-05-14 02:06:48,markdavidson,@Monkeylover35 Yeah. I had to get away for a b...,3,02:06:48,2009-05-14,93,0,False,...,96.69,3.4,0.50,0.500000,0.290,0.710,0.0,0.5994,2,0.8750
2,1,2009-05-04 07:14:48,Maekii,The World is just amazing!,0,07:14:48,2009-05-04,27,0,False,...,100.24,2.0,0.75,0.900000,0.506,0.494,0.0,0.6239,7,0.5625
3,0,2009-06-25 02:35:25,MissElisee,Just picked my sister up from work. she's al...,3,02:35:25,2009-06-25,100,0,False,...,95.17,4.0,0.00,0.000000,0.000,1.000,0.0,0.0000,2,0.8750
4,0,2009-06-19 13:40:45,MissK20,I don't think changing my last name is meant t...,4,13:40:45,2009-06-19,52,0,False,...,102.61,4.4,0.00,0.066667,0.000,1.000,0.0,0.0000,13,0.1875


In [11]:
#It is kind of forced but we can try doing similar thing with Weekday
df['skewed_week_dist'] = df['Weekday'].apply(lambda x: (2 - x) / 2 if x < 2 else (x - 2) / 4)
df.head()

,Target,Date,User,Text,Weekday,Time,Full_date,Length,Hashtags,HasHashtags,...,GFI,Polarity_TB,Subjectivity_TB,Positive_VADER,Neutral_VADER,Negative_VADER,Compound_VADER,Hour,skewed_hour_dist,skewed_week_dist
0,0,2009-06-25 00:52:47,timofuchs,@grazzly maaaaaate will you be at Glasto? Not ...,3,00:52:47,2009-06-25,67,0,False,...,2.2,0.00,0.000000,0.000,1.000,0.0,0.0000,0,1.0000,0.25
1,1,2009-05-14 02:06:48,markdavidson,@Monkeylover35 Yeah. I had to get away for a b...,3,02:06:48,2009-05-14,93,0,False,...,3.4,0.50,0.500000,0.290,0.710,0.0,0.5994,2,0.8750,0.25
2,1,2009-05-04 07:14:48,Maekii,The World is just amazing!,0,07:14:48,2009-05-04,27,0,False,...,2.0,0.75,0.900000,0.506,0.494,0.0,0.6239,7,0.5625,1.00
3,0,2009-06-25 02:35:25,MissElisee,Just picked my sister up from work. she's al...,3,02:35:25,2009-06-25,100,0,False,...,4.0,0.00,0.000000,0.000,1.000,0.0,0.0000,2,0.8750,0.25
4,0,2009-06-19 13:40:45,MissK20,I don't think changing my last name is meant t...,4,13:40:45,2009-06-19,52,0,False,...,4.4,0.00,0.066667,0.000,1.000,0.0,0.0000,13,0.1875,0.50


In [12]:
#We have seen that after certain date (2009-05-29 07:33:45) sentiment of all tweets is negative. Of course it is a feature
#strictly specific to our data and according to common sense shouldn't be used in the model if we would like to predict the sentiment
#of any text. However if testing is going to be done on our set alone it is a meaningful piece of information.
y = df[df['Target'] == 1]['Full_date'].max()
df['is_after_certain_day'] = df['Full_date'].apply(lambda x: 0 if x <= y else 1)
df.head()

,Target,Date,User,Text,Weekday,Time,Full_date,Length,Hashtags,HasHashtags,...,Polarity_TB,Subjectivity_TB,Positive_VADER,Neutral_VADER,Negative_VADER,Compound_VADER,Hour,skewed_hour_dist,skewed_week_dist,is_after_certain_day
0,0,2009-06-25 00:52:47,timofuchs,@grazzly maaaaaate will you be at Glasto? Not ...,3,00:52:47,2009-06-25,67,0,False,...,0.00,0.000000,0.000,1.000,0.0,0.0000,0,1.0000,0.25,1
1,1,2009-05-14 02:06:48,markdavidson,@Monkeylover35 Yeah. I had to get away for a b...,3,02:06:48,2009-05-14,93,0,False,...,0.50,0.500000,0.290,0.710,0.0,0.5994,2,0.8750,0.25,0
2,1,2009-05-04 07:14:48,Maekii,The World is just amazing!,0,07:14:48,2009-05-04,27,0,False,...,0.75,0.900000,0.506,0.494,0.0,0.6239,7,0.5625,1.00,0
3,0,2009-06-25 02:35:25,MissElisee,Just picked my sister up from work. she's al...,3,02:35:25,2009-06-25,100,0,False,...,0.00,0.000000,0.000,1.000,0.0,0.0000,2,0.8750,0.25,1
4,0,2009-06-19 13:40:45,MissK20,I don't think changing my last name is meant t...,4,13:40:45,2009-06-19,52,0,False,...,0.00,0.066667,0.000,1.000,0.0,0.0000,13,0.1875,0.50,1


In [13]:
#Thaks to one of correlation matrices from EDA (or to having at least 1 brain cell) we know that the number of hashtags, mentions or
#exclamation marks is highly correlated with its binary counterpart quantifier. From the same analysis we see that their correlation with 
# target is marginal however mentions and exclamation marks seem to have more to say than hashtags, that is why we 
#will leave out hashtags but not the other two.
def has_men(text):
    for c in text:
        if c == '@':
            return 1
    return 0


def has_exc(text):
    for c in text:
        if c == '!':
            return 1
        return 0


df['has_mentions'] = df['Text'].apply(has_men)
df['has_exclamation_marks'] = df['Text'].apply(has_exc)
df.head()

,Target,Date,User,Text,Weekday,Time,Full_date,Length,Hashtags,HasHashtags,...,Positive_VADER,Neutral_VADER,Negative_VADER,Compound_VADER,Hour,skewed_hour_dist,skewed_week_dist,is_after_certain_day,has_mentions,has_exclamation_marks
0,0,2009-06-25 00:52:47,timofuchs,@grazzly maaaaaate will you be at Glasto? Not ...,3,00:52:47,2009-06-25,67,0,False,...,0.000,1.000,0.0,0.0000,0,1.0000,0.25,1,1,0
1,1,2009-05-14 02:06:48,markdavidson,@Monkeylover35 Yeah. I had to get away for a b...,3,02:06:48,2009-05-14,93,0,False,...,0.290,0.710,0.0,0.5994,2,0.8750,0.25,0,1,0
2,1,2009-05-04 07:14:48,Maekii,The World is just amazing!,0,07:14:48,2009-05-04,27,0,False,...,0.506,0.494,0.0,0.6239,7,0.5625,1.00,0,0,0
3,0,2009-06-25 02:35:25,MissElisee,Just picked my sister up from work. she's al...,3,02:35:25,2009-06-25,100,0,False,...,0.000,1.000,0.0,0.0000,2,0.8750,0.25,1,0,0
4,0,2009-06-19 13:40:45,MissK20,I don't think changing my last name is meant t...,4,13:40:45,2009-06-19,52,0,False,...,0.000,1.000,0.0,0.0000,13,0.1875,0.50,1,0,0


In [14]:
#And now for the main course tf-idf, we will set a limit o a 100 words so as not to everything too high in computation cost.
from sklearn.feature_extraction.text import TfidfVectorizer

text_data = df['Text'].tolist()

#We shall also remove certain english stop words because they are very frequent but do not carry much sentiment.
vectoriser = TfidfVectorizer(max_features=50, lowercase=True, stop_words='english')
tfidf_matrix = vectoriser.fit_transform(text_data)
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectoriser.get_feature_names_out())
# df = pd.concat([df, tfidf_df], axis=1)
df.head()

,Target,Date,User,Text,Weekday,Time,Full_date,Length,Hashtags,HasHashtags,...,Positive_VADER,Neutral_VADER,Negative_VADER,Compound_VADER,Hour,skewed_hour_dist,skewed_week_dist,is_after_certain_day,has_mentions,has_exclamation_marks
0,0,2009-06-25 00:52:47,timofuchs,@grazzly maaaaaate will you be at Glasto? Not ...,3,00:52:47,2009-06-25,67,0,False,...,0.000,1.000,0.0,0.0000,0,1.0000,0.25,1,1,0
1,1,2009-05-14 02:06:48,markdavidson,@Monkeylover35 Yeah. I had to get away for a b...,3,02:06:48,2009-05-14,93,0,False,...,0.290,0.710,0.0,0.5994,2,0.8750,0.25,0,1,0
2,1,2009-05-04 07:14:48,Maekii,The World is just amazing!,0,07:14:48,2009-05-04,27,0,False,...,0.506,0.494,0.0,0.6239,7,0.5625,1.00,0,0,0
3,0,2009-06-25 02:35:25,MissElisee,Just picked my sister up from work. she's al...,3,02:35:25,2009-06-25,100,0,False,...,0.000,1.000,0.0,0.0000,2,0.8750,0.25,1,0,0
4,0,2009-06-19 13:40:45,MissK20,I don't think changing my last name is meant t...,4,13:40:45,2009-06-19,52,0,False,...,0.000,1.000,0.0,0.0000,13,0.1875,0.50,1,0,0


In [15]:
import re
import emoji
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

custom_stopwords = set(stopwords.words('english')) - {'no', 'not', 'nor', "isn't", "wasn't", "aren't", "don't",
                                                      "didn't", "cannot", "couldn't", "shouldn't"}


def preprocess_tweet(tweet):
    tweet = str(tweet).lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    tweet = re.sub(r'\@\w+|\#', '', tweet)
    tweet = re.sub(r'\W', ' ', tweet)
    tweet = emoji.demojize(tweet, delimiters=("", " "))
    tokens = word_tokenize(tweet)
    tokens = [word for word in tokens if word not in custom_stopwords]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)


df['ProcessedText'] = df['Text'].apply(preprocess_tweet)

In [16]:
from gensim.models import Word2Vec

texts = df['ProcessedText'].tolist()

tokenized_texts = [text.split() for text in texts]

word2vec_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)


def tweet_vector(tweet, model):
    words = tweet.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    else:
        return np.mean(word_vectors, axis=0)


df["w2v_feature"] = df['ProcessedText'].apply(lambda tweet: tweet_vector(tweet, word2vec_model))

In [17]:
df.head()

,Target,Date,User,Text,Weekday,Time,Full_date,Length,Hashtags,HasHashtags,...,Negative_VADER,Compound_VADER,Hour,skewed_hour_dist,skewed_week_dist,is_after_certain_day,has_mentions,has_exclamation_marks,ProcessedText,w2v_feature
0,0,2009-06-25 00:52:47,timofuchs,@grazzly maaaaaate will you be at Glasto? Not ...,3,00:52:47,2009-06-25,67,0,False,...,0.0,0.0000,0,1.0000,0.25,1,1,0,maaaaaate glasto not going blissfeilds,"[-0.06496561, 0.48627812, -0.038093906, 0.0815..."
1,1,2009-05-14 02:06:48,markdavidson,@Monkeylover35 Yeah. I had to get away for a b...,3,02:06:48,2009-05-14,93,0,False,...,0.0,0.5994,2,0.8750,0.25,0,1,0,yeah get away bit time time healthy,"[-0.32092136, 0.73897886, -0.25996143, 0.02978..."
2,1,2009-05-04 07:14:48,Maekii,The World is just amazing!,0,07:14:48,2009-05-04,27,0,False,...,0.0,0.6239,7,0.5625,1.00,0,0,0,world amazing,"[-0.3094858, 0.30486667, 0.10236198, 0.0075460..."
3,0,2009-06-25 02:35:25,MissElisee,Just picked my sister up from work. she's al...,3,02:35:25,2009-06-25,100,0,False,...,0.0,0.0000,2,0.8750,0.25,1,0,0,picked sister work always grumpz work think ne...,"[-0.019012148, 0.7563662, -0.23994003, -0.0061..."
4,0,2009-06-19 13:40:45,MissK20,I don't think changing my last name is meant t...,4,13:40:45,2009-06-19,52,0,False,...,0.0,0.0000,13,0.1875,0.50,1,0,0,think changing last name meant,"[0.16587625, 0.8152086, -0.15254061, -0.115974..."


In [18]:
import nltk
from nltk.corpus import stopwords
import tiktoken
import numpy as np

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
stop_words = set(stopwords.words('english'))


def get_weighted_embedding(text):
    tokens = nltk.word_tokenize(text)
    tagged_tokens = nltk.pos_tag(tokens)
    embeddings = []
    for word, tag in tagged_tokens:
        if word in stop_words:
            continue
        embedding = encoding.encode(word)
        weight = 1.5 if tag.startswith('JJ') else 1.0
        embeddings.append(np.average(embedding) * weight)

    if len(embeddings) == 0:
        return 0
    weighted_embeddings = np.average(embeddings, axis=0)
    return weighted_embeddings


df["tt_embedding_weighted"] = np.array([get_weighted_embedding(text) for text in texts])

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

texts = df['ProcessedText']
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = tfidf_vectorizer.fit_transform(texts)
tfidf_array = tfidf_matrix.toarray()
feature_names = tfidf_vectorizer.get_feature_names_out()


def top_n_words(row, feature_names, n=3):
    top_n_idx = np.argsort(row)[-n:]
    top_n_values = [feature_names[i] for i in top_n_idx]
    return top_n_values


df[["Word_1", "Word_2", "Word_3"]] = [top_n_words(row, feature_names) for row in tfidf_array]
df

,Target,Date,User,Text,Weekday,Time,Full_date,Length,Hashtags,HasHashtags,...,skewed_week_dist,is_after_certain_day,has_mentions,has_exclamation_marks,ProcessedText,w2v_feature,tt_embedding_weighted,Word_1,Word_2,Word_3
0,0,2009-06-25 00:52:47,timofuchs,@grazzly maaaaaate will you be at Glasto? Not ...,3,00:52:47,2009-06-25,67,0,False,...,0.25,1,1,0,maaaaaate glasto not going blissfeilds,"[-0.06496561, 0.48627812, -0.038093906, 0.0815...",5738.966667,yummy,not,going
1,1,2009-05-14 02:06:48,markdavidson,@Monkeylover35 Yeah. I had to get away for a b...,3,02:06:48,2009-05-14,93,0,False,...,0.25,0,1,0,yeah get away bit time time healthy,"[-0.32092136, 0.73897886, -0.25996143, 0.02978...",22649.571429,away,bit,time
2,1,2009-05-04 07:14:48,Maekii,The World is just amazing!,0,07:14:48,2009-05-04,27,0,False,...,1.00,0,0,0,world amazing,"[-0.3094858, 0.30486667, 0.10236198, 0.0075460...",9254.500000,yummy,world,amazing
3,0,2009-06-25 02:35:25,MissElisee,Just picked my sister up from work. she's al...,3,02:35:25,2009-06-25,100,0,False,...,0.25,1,0,0,picked sister work always grumpz work think ne...,"[-0.019012148, 0.7563662, -0.23994003, -0.0061...",15720.750000,wont,sister,work
4,0,2009-06-19 13:40:45,MissK20,I don't think changing my last name is meant t...,4,13:40:45,2009-06-19,52,0,False,...,0.50,1,0,0,think changing last name meant,"[0.16587625, 0.8152086, -0.15254061, -0.115974...",17907.800000,last,name,meant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,2009-05-09 22:01:05,SoupAirCool,i wanna see you,5,22:01:05,2009-05-09,16,0,False,...,0.75,0,0,0,wan na see,"[-0.4519527, 0.9470214, 0.0039422116, 0.056109...",8191.333333,na,see,wan
99996,0,2009-06-19 18:28:38,AFx3,awee i want more thunderstorms..,4,18:28:38,2009-06-19,33,0,False,...,0.50,1,0,0,awee want thunderstorm,"[-0.57095385, 0.48754776, 0.13556938, 0.185115...",22394.611111,fml,yummy,want
99997,0,2009-06-15 18:51:07,gracethegal,"@poison_ive3 Sorry, I just feel like it's the...",0,18:51:07,2009-06-15,122,0,False,...,1.00,1,1,0,sorry feel like start trip get really excited ...,"[-0.24089342, 0.79467255, -0.46386936, -0.0194...",29204.409091,trip,trouble,get
99998,1,2009-05-01 23:56:02,Kiralany,prepping for my engagement shoot tomorrow with...,4,23:56:02,2009-05-01,87,0,False,...,0.50,0,0,0,prepping engagement shoot tomorrow meredith am...,"[-0.17601845, 0.30991325, -0.10451006, 0.07090...",19646.666667,tomorrow,fun,shoot


In [20]:
def get_tiktoken_embedding(text):
    embeddings = encoding.encode(text)
    if len(embeddings) == 0:
        return 0, 0, 0
    return np.mean(embeddings, axis=0), np.max(embeddings, axis=0), np.min(embeddings, axis=0)


for i in range(1, 4):
    words = df[f"Word_{i}"]
    df[[f"Word_{i}_embedding_mean", f"Word_{i}_embedding_max", f"Word_{i}_embedding_min"]] = np.array(
        [get_tiktoken_embedding(word) for word in words])

In [21]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 10
lda = LatentDirichletAllocation(n_components=n_topics, random_state=0)
lda.fit(tfidf_matrix)
topic_distribution = lda.transform(tfidf_matrix)
dominant_topic = np.argmax(topic_distribution, axis=1)
df['DominantTopic'] = dominant_topic

In [22]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')


def generate_sentence_embeddings(texts, model):
    return model.encode(texts, batch_size=32, show_progress_bar=True)


texts = df['ProcessedText'].tolist()
embeddings = generate_sentence_embeddings(texts, model)
embeddings_list = embeddings.tolist()
df['sentence_embeddings'] = embeddings_list
for i in range(len(embeddings[0])):
    df[f'embedding_{i}'] = df['sentence_embeddings'].apply(lambda x: x[i])


/home/adam/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

/tmp/ipykernel_46189/1503368725.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'embedding_{i}'] = df['sentence_embeddings'].apply(lambda x: x[i])
/tmp/ipykernel_46189/1503368725.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'embedding_{i}'] = df['sentence_embeddings'].apply(lambda x: x[i])
/tmp/ipykernel_46189/1503368725.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once

In [23]:
import spacy

nlp = spacy.load("en_core_web_sm")


def get_named_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]


df['NamedEntities'] = df['Text'].apply(get_named_entities)
all_labels = set([label for sublist in df['NamedEntities'] for _, label in sublist])

for label in all_labels:
    df[f'Count_{label}'] = 0


def update_label_counts(row):
    label_counts = {label: 0 for label in all_labels}

    for _, label in row['NamedEntities']:
        label_counts[label] += 1

    for label, count in label_counts.items():
        row[f'Count_{label}'] = count

    return row


df = df.apply(update_label_counts, axis=1)

/tmp/ipykernel_46189/2425607403.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['NamedEntities'] = df['Text'].apply(get_named_entities)
/tmp/ipykernel_46189/2425607403.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Count_{label}'] = 0
/tmp/ipykernel_46189/2425607403.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented f

In [24]:
for i in range(word2vec_model.vector_size):
    df[f'w2v_{i}'] = df['w2v_feature'].apply(lambda x: x[i])

df.drop(['w2v_feature', 'NamedEntities'], axis=1, inplace=True)

/tmp/ipykernel_46189/3035963235.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'w2v_{i}'] = df['w2v_feature'].apply(lambda x: x[i])
/tmp/ipykernel_46189/3035963235.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'w2v_{i}'] = df['w2v_feature'].apply(lambda x: x[i])
/tmp/ipykernel_46189/3035963235.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instea

In [25]:
df.to_csv("./data/tweets_features.csv", index=False)

In [26]:
df.head()

,Target,Date,User,Text,Weekday,Time,Full_date,Length,Hashtags,HasHashtags,...,w2v_90,w2v_91,w2v_92,w2v_93,w2v_94,w2v_95,w2v_96,w2v_97,w2v_98,w2v_99
0,0,2009-06-25 00:52:47,timofuchs,@grazzly maaaaaate will you be at Glasto? Not ...,3,00:52:47,2009-06-25,67,0,False,...,0.426355,0.105600,0.099900,0.141384,0.683017,0.307975,0.141498,-0.470852,-0.146050,0.013814
1,1,2009-05-14 02:06:48,markdavidson,@Monkeylover35 Yeah. I had to get away for a b...,3,02:06:48,2009-05-14,93,0,False,...,0.893100,0.385402,-0.035950,-0.465414,1.386158,0.869907,0.718067,-0.696911,-0.041971,-0.195380
2,1,2009-05-04 07:14:48,Maekii,The World is just amazing!,0,07:14:48,2009-05-04,27,0,False,...,0.590157,0.242431,-0.339337,0.507457,1.383398,1.200003,0.258323,-0.451583,0.037492,0.057117
3,0,2009-06-25 02:35:25,MissElisee,Just picked my sister up from work. she's al...,3,02:35:25,2009-06-25,100,0,False,...,0.741138,0.014045,-0.037039,-0.300930,1.460556,0.542765,0.764829,-0.621368,-0.208227,-0.209104
4,0,2009-06-19 13:40:45,MissK20,I don't think changing my last name is meant t...,4,13:40:45,2009-06-19,52,0,False,...,0.621242,0.201937,-0.061574,0.177444,1.245472,0.712410,0.349455,-0.450614,0.096795,0.056020
